In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab/ai_scholar/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
## @title Install Packages and Load Model
# !pip install -q tf-nightly
# !pip install -q tf-models-nightly
# from official.modeling import tf_utils
!pip install -q tf-models-official==2.4.0
!pip install -q -U --no-cache-dir gdown --pre
from official import nlp
# from official.nlp import bert
# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks
!pip install -q transformers
!pip install -q sentencepiece
# !pip uninstall transformers
# !pip install transformers==3.5
# Load Huggingface transformers
from transformers import AlbertConfig, AlbertModel,AlbertTokenizer,TFAlbertForTokenClassification
from transformers import BatchEncoding
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import pickle
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from transformers import BatchEncoding
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import XLNetTokenizer, TFXLNetModel,XLNetConfig
import numpy as np
import pickle
from tensorflow.keras.models import model_from_json
# from random_user_agent.user_agent import UserAgent
!gdown --id 1MLb357lkypd7GQIv-vKXB_gqSqBLxFkK
!gdown --id 1839biKxQGvmCISDCXKWQmC1EU5eBUTiB
!gdown --id 1sSKsdjQfrFGcRwmc_rD-obM8cqj4Awsb
model_name = 'xlnet-large-cased'
# Load transformers config and set output_hidden_states to False
config = XLNetConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load xlnet tokenizer
tokenizer = XLNetTokenizer.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
json_file = open(GOOGLE_DRIVE_PATH+'data/nationality/my_best_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(GOOGLE_DRIVE_PATH+"data/nationality/my_best_model.hdf5")
print("Loaded model from disk")
# Set an optimizer
epochs = 25
batch_size = 128
eval_batch_size = 128
final_data=pd.read_csv(GOOGLE_DRIVE_PATH+'data/nationality/country_list.csv')
train_data_size = len(final_data)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)
# creates an optimizer with learning rate schedule
# Set an optimizer
# optimizer = Adam(
#     learning_rate=5e-05,
#     epsilon=1e-08,
#     decay=0.01,
#     clipnorm=1.0)
# Set loss and metrics
loss = {'Country': CategoricalCrossentropy(from_logits = True)}
metric = {'Country': CategoricalAccuracy('accuracy')}
# Compile the model
loaded_model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
df = pd.read_csv(GOOGLE_DRIVE_PATH+'out/unclassified_gender.csv')
full_list = df['name'].tolist()

In [ ]:
# #@title Name Country prediction

# Full_Name= "ameer tamoor khan" #@param {type:"string"}

from collections import defaultdict
national_dict = defaultdict(int)
for full_name in full_list:
  a = tokenizer.encode(
      text= full_name,
      add_special_tokens=True,
      max_length=10,
      truncation=True,
      padding=True, 
      return_tensors='tf',
      return_token_type_ids = False,
      return_attention_mask = False,
      verbose = True)
  a_as_vector = tf.reshape(a, [-1])
  zero_padding = tf.zeros([1 * 10] - tf.shape(a_as_vector), dtype=a.dtype)
  a_reshape=tf.concat([a_as_vector, zero_padding],0)
  a_reshape=tf.reshape(a_reshape,[1,10])
  d=loaded_model.predict(a_reshape)
  c=np.argmax(d['Country'])
  selec=final_data[final_data.Country==c]
  national_dict[selec['Country_belongs'].iloc[0]] += 1
national_dict

In [ ]:
national_dict